
An attempt to measure suprise in text using adapters

https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np
import pandas as pd

In [3]:
import os

import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# from peft import LoraConfig, get_peft_model


# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [30]:
from bs_writing_detector.data.load_md import load_md_df
from pathlib import Path

df = load_md_df(Path("../samples/"))

2025-01-06 11:35:53.921 | DEBUG    | bs_writing_detector.data.load_md:load_md:10 - ../samples/2024_anthropic_palintir.md
2025-01-06 11:35:53.921 | DEBUG    | bs_writing_detector.data.load_md:load_md:10 - ../samples/2024_arxiv_meh.md
2025-01-06 11:35:53.922 | DEBUG    | bs_writing_detector.data.load_md:load_md:10 - ../samples/2024_bob_fanfic.md
2025-01-06 11:35:53.922 | DEBUG    | bs_writing_detector.data.load_md:load_md:10 - ../samples/2024_bob_fanfic2.md
2025-01-06 11:35:53.922 | DEBUG    | bs_writing_detector.data.load_md:load_md:10 - ../samples/2024_deliberative_alignment.md
2025-01-06 11:35:53.940 | DEBUG    | bs_writing_detector.data.load_md:load_md:10 - ../samples/2024_gpt4_fake_paper.md
2025-01-06 11:35:53.941 | DEBUG    | bs_writing_detector.data.load_md:load_md:10 - ../samples/2024_gwern_reddit.md
2025-01-06 11:35:53.941 | DEBUG    | bs_writing_detector.data.load_md:load_md:10 - ../samples/2024_how_to_focus.md
2025-01-06 11:35:53.942 | DEBUG    | bs_writing_detector.data.load_

In [ ]:
max_summary_frac    = 0.1
max_chars = 2000

## Helpers

In [9]:
from dotenv import load_dotenv

load_dotenv() 

True

In [10]:
import os
from openai import OpenAI
from anycache import anycache

cache_file = "../.anycache"

# to clear
import shutil
shutil.rmtree(cache_file, ignore_errors=True)

@anycache(cachedir=cache_file)
def summize_gpt4(text):
    client = OpenAI()
    content = f"Make a tl;dr of this text in <280 chars.\n\n## Text\n\n{text}\n\n## Instruction\n\nMake a tl;dr of this text in <280 chars. Start with the most important, as extra text will be discarded :\n\ntl;dr:"
    chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="gpt-4",
        )
    r = chat_completion.choices[0].message.content
    return r

@anycache(cachedir=cache_file)
def summarize_gpt4b(text):
    l = int(len(text)*max_summary_frac)
    # print(l)
    client = OpenAI()
    inst = "We aim to compress then reconstruct a text. First lets do the compression. In short hand, record the information needed to reconstruct the text (type of document, writing style, suprising contenxt, etc). You have <{l} chars. Start with the most important, as extra text will be discarded"
    content = f"{inst}\n\n## Text\n\n{text}\n\n## Instruction\n\n{inst}:\n\ntl;dr:"
    chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
            model="gpt-4",
        )
    # print(content)
    r = chat_completion.choices[0].message.content
    return r[:l], r[l:]

r, _ = summarize_gpt4b(samples[1]["content"])
print(r), print(_)

Academic paper titled "TradingAgents: Multi-Agents LLM Financial Trading Framework". Authors propose new framework using large language model (LLM) powered agents for collaborative finance trading. Mu
lti-agent system mirrors trading firm dynamics with analysts & traders roles, improving performance. Highlights advantages over traditional models.


(None, None)

In [11]:
r = summize_gpt4(samples[1]["content"])
print(r)

TradingAgents is a novel multi-agent framework for financial trading powered by large language models (LLMs), aiming to replicate the dynamics of real-world trading firms to enhance trading performance.


In [12]:
from bs_writing_detector.data.load_md import load_md_df
from bs_writing_detector.metrics.ppx import perplexity_compute

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM, AutoConfig, PreTrainedTokenizerBase, PreTrainedTokenizer, GPTQConfig, BitsAndBytesConfig

def load_model(model_name):
    trust_remote_code = True
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=trust_remote_code)
    # print(config)
    if config.quantization_config is not None:
        config.quantization_config['disable_exllama'] = True
        if 'use_exllama' in config.quantization_config:
            del config.quantization_config['use_exllama']
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=trust_remote_code, 
                                                 config=config,
                                                 )
    return model, tokenizer


In [14]:
import gc

def clear_mem():
    torch.cuda.empty_cache()
    gc.collect()

## Results

In [15]:
models = [
    # "TheBloke/phi-2-GPTQ",
    # "TheBloke/Llama-2-7B-GPTQ",
    # "TheBloke/Llama-2-13B-GPTQ",
    # "TheBloke/Mistral-7B-v0.1-GPTQ",
    #  "unsloth/Llama-3.2-1B",
     "unsloth/Llama-3.2-1B-bnb-4bit",
]
# model_name = "unsloth/Llama-3.2-1B"
# model_name = "unsloth/Llama-3.2-1B-bnb-4bit"

In [17]:
from tqdm.auto import tqdm

summaries = {}
data = []
for model_name in models:
    model, tokenizer = load_model(model_name)
    print(model_name)
    for i in tqdm(range(len(df))):
        sample = df.iloc[i]
        if sample['title'] not in summaries:
            summaries[sample['title']] = summarize_gpt4b(sample['content'])[0]
        summary = summaries[sample['title']]

        # before 
        s1 = sample['content']
        before = perplexity_compute(data=s1, model=model, tokenizer=tokenizer, device='cuda')['nlls'][0]

        n_tokens = len(tokenizer(s1)['input_ids'])

        # after 
        s2 = f"""
        High level summary: {summary}

Text:
{sample['content']}
        """

        # TODO add attn mask
        after = perplexity_compute(data=s2, model=model, tokenizer=tokenizer, device='cuda')['nlls'][0][-n_tokens:]

        print(f"{before.mean()} {after.mean()} | {sample['title']}")
        data.append(dict(
            before_mean=before.mean(), 
            before_sum=before.sum().item(),
            before_std=before.std(),
            before_min=before.min(),
            before_max=before.max(),
            before_ppx=np.exp(before.mean().item()),
                         after_mean=after.mean(),
                         after_sum=after.sum().item(),
                            after_std=after.std(),
                            after_min=after.min(),
                            after_max=after.max(), 
                            after_ppx=np.exp(after.mean().item()),
                         model=model_name, 
                         summary=summary,
                        #  sample=sample['title'],
                        #  in_training=sample['in_training'], 
                         len=len(sample['content']),
                         n_tokens=n_tokens,
                         **sample
                         ),

        )


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method', 'disable_exllama']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


unsloth/Llama-3.2-1B-bnb-4bit


  0%|          | 0/30 [00:00<?, ?it/s]

2.731894616981088 2.4346583814670644 | Anthropic and Palantir Partner to Bring Claude AI Models to AWS for U.S. Government Intelligence and Defense Operations
3.018628165803172 2.787076804860793 | TradingAgents: Multi-Agents LLM Financial Trading Framework
3.1768678330086373 3.0573941246602545 | Flower Crowns and Furry Mishaps by MyPalAI
3.7198927586704955 3.619981446455229 | Paradox's Box (Bobiverse) by Mark4man
3.22479815838576 3.0429821194746554 | Deliberative Alignment: Reasoning Enables Safer Language Models
2.341998207471676 2.09474255727685 | fake ai hoax paper made up by gpt-4
3.66733124948317 3.466211730036242 | Hardware Hedging Against Scaling Regime Shifts (self.mlscaling)
1.7733189346720872 1.6321287018846171 | How to Focus
2.7362908764318985 2.4942711856630115 | Deontic Explorations In "Paying To Talk To Slaves"
2.9917198510817538 2.809688214010793 | By default, capital will matter more than ever after AGI
2.8518650030297987 2.7567292506743093 | The Plan - 2024 Update
2.52

In [18]:
# results
df2 = pd.DataFrame(data).set_index('title')
df2 = df2.query('in_training == False')
df2

,before_mean,before_sum,before_std,before_min,before_max,before_ppx,after_mean,after_sum,after_std,after_min,...,model,summary,len,n_tokens,f,content,url,novelty,date,in_training
title,,,,,,,,,,,,,,,,,,,,,
Anthropic and Palantir Partner to Bring Claude AI Models to AWS for U.S. Government Intelligence and Defense Operations,2.731895,1046.315638,2.671276,0.000052,15.757812,15.361965,2.434658,934.908818,2.503916,0.000028,...,unsloth/Llama-3.2-1B-bnb-4bit,"Anthropic, Palantir (PLTR), AWS partnership an...",2000,384,../samples/2024_anthropic_palintir.md,Anthropic and Palantir Technologies Inc. (NYSE...,https://investors.palantir.com/news-details/20...,0.200000,2024-07-11 00:00:00+00:00,False
TradingAgents: Multi-Agents LLM Financial Trading Framework,3.018628,1062.557114,2.947409,0.000241,15.257812,20.463200,2.787077,983.838112,2.807337,0.000211,...,unsloth/Llama-3.2-1B-bnb-4bit,"Academic paper titled ""TradingAgents: Multi-Ag...",2000,353,../samples/2024_arxiv_meh.md,TradingAgents: Multi-Agents LLM Financial Trad...,https://arxiv.org/html/2412.20138v1,0.150000,2024-12-28 00:00:00+00:00,False
Flower Crowns and Furry Mishaps by MyPalAI,3.176868,1528.073428,2.790151,0.000415,15.187500,23.971553,3.057394,1473.663968,2.680333,0.000613,...,unsloth/Llama-3.2-1B-bnb-4bit,"Sci-fi story set in Delta Eridani. Character, ...",2000,482,../samples/2024_bob_fanfic.md,"Bob, April 2222 Delta Eridani\n\n \n\nThere ar...",https://archiveofourown.org/works/61614673/cha...,0.300000,2024-12-24 00:00:00+00:00,False
Deliberative Alignment: Reasoning Enables Safer Language Models,3.224798,1254.446484,3.099538,0.000083,16.718750,25.148498,3.042982,1186.763027,2.987466,0.000122,...,unsloth/Llama-3.2-1B-bnb-4bit,Research paper on Deliberative Alignment for s...,2000,390,../samples/2024_deliberative_alignment.md,# Deliberative Alignment: Reasoning Enables Sa...,https://arxiv.org/html/2412.16339v1,0.600000,2024-12-20 00:00:00+00:00,False
fake ai hoax paper made up by gpt-4,2.341998,859.513342,2.409817,0.000074,23.328125,10.402001,2.094743,770.865261,2.315665,0.000097,...,unsloth/Llama-3.2-1B-bnb-4bit,"Academic paper. Title: ""Deconstructing Binarie...",2000,368,../samples/2024_gpt4_fake_paper.md,"Title: ""Deconstructing Binaries: Interrogating...",None,0.000000,2024-01-01 00:00:00+00:00,False
Hardware Hedging Against Scaling Regime Shifts (self.mlscaling),3.667331,1591.621762,3.091328,0.001032,15.445312,39.147292,3.466212,1507.802103,3.064746,0.000542,...,unsloth/Llama-3.2-1B-bnb-4bit,Big tech investing heavily in GPUs for paralle...,2000,435,../samples/2024_gwern_reddit.md,Hyperscalers are investing heavily in AMD/Nvid...,https://old.reddit.com/r/mlscaling/comments/1e...,1.000000,2024-08-21 00:00:00+00:00,False
How to Focus,1.773319,913.259251,2.641230,0.000018,14.210938,5.890371,1.632129,842.178410,2.544260,0.000024,...,unsloth/Llama-3.2-1B-bnb-4bit,"Book titled ""How to Focus: A Monastic Guide fo...",2000,516,../samples/2024_how_to_focus.md,# HOW TO FOCUS\n\n\n\n\n\nANCIENT WISDOM FOR M...,https://github.com/wassname/stoic_markdown/blo...,0.500000,2024-06-01 00:00:00+00:00,False
"Deontic Explorations In ""Paying To Talk To Slaves""",2.736291,1203.967986,2.828962,0.000119,13.765625,15.429648,2.494271,1099.973593,2.668889,0.000089,...,unsloth/Llama-3.2-1B-bnb-4bit,AI and human engage in a Kantian ethics dialog...,2000,441,../samples/2024_lesswrong_slop.md,HER|Jenny|🤔: I want to read a dialogue between...,https://www.lesswrong.com/posts/Rk2o8hjYmjENH8...,0.100000,2024-04-12 00:00:00+00:00,False
"By default, capital will matter more than ever after AGI",2.991720,1211.646540,2.607116,0.000087,15.765625,19.919912,2.809688,1140.733415,2.478602,0.000094,...,unsloth/Llama-3.2-1B-bnb-4bit,AI replacing human labor changes production fa...,2000,406,../samples/2024_lw_by-default-capital-will-mat...,**Edited to add*****:** The main takeaway of t...,https://www.lesswrong.com/posts/KFFaKu27FNugCH...,0.906388,2024-12-30 19:47:19.838000+00:00,False


In [19]:
# QC summary
df2.summary.str.len().describe()

count     27.000000
mean     195.925926
std       18.097866
min      106.000000
25%      200.000000
50%      200.000000
75%      200.000000
max      200.000000
Name: summary, dtype: float64

In [20]:
for n,d in df2.groupby("model"):
    for stat in ['mean', 'std', 'min', 'max', 'ppx']:
        df2[f"improvement%_{stat}"] = (df2[f"before_{stat}"] - df2[f"after_{stat}"]) / df2[f"before_{stat}"]
        df2[f"improvement_{stat}"] = (df2[f"before_{stat}"] - df2[f"after_{stat}"])
        df2[f"summarizable_{stat}"] = df2[f"improvement_{stat}"]  > 1
        df2[f"summarizable2_{stat}"] = df2[f"improvement%_{stat}"]  > 0.05
        df2[f'suprising_{stat}'] = df2[f"before_{stat}"] > 15
        df2[f'BS_{stat}'] = ~df2[f"summarizable_{stat}"] | ~df2[f'suprising_{stat}']


    print(n)
    # d = d[[ 'before', 'after', "improvement", "improvement%", 'suprising', 'summarizable', 'summarizable2', 'novelty' ]].sort_values("improvement%", ascending=True)
    # print(d.to_markdown())
    # display(d)

    # TODO turn into a single metric, correlate with novelty label
    r = df2.select_dtypes(include=np.number).corr()['novelty'].abs().sort_values()
    display(r)

unsloth/Llama-3.2-1B-bnb-4bit


after_min            0.000692
improvement%_min     0.080467
before_min           0.114827
after_max            0.165648
improvement_min      0.179910
len                  0.222931
improvement_ppx      0.259635
improvement_mean     0.260877
improvement%_ppx     0.264841
before_max           0.357355
improvement_max      0.367002
before_std           0.367073
improvement%_max     0.369768
n_tokens             0.404075
before_mean          0.436573
improvement%_mean    0.442458
before_ppx           0.454817
improvement_std      0.457283
after_ppx            0.461407
after_std            0.464671
after_mean           0.470662
improvement%_std     0.482445
before_sum           0.574461
after_sum            0.580702
novelty              1.000000
Name: novelty, dtype: float64

In [28]:
df2[[ 'improvement%_mean', 'novelty', 'date']].sort_values("improvement%_mean", ascending=True)

,improvement%_mean,novelty,date
title,,,
2024 in AI predictions,0.026404,0.789190,2025-01-02 02:09:13.093000+00:00
The Plan - 2024 Update,0.033359,0.785170,2024-12-31 19:29:23.013000+00:00
The Intelligence Curse,0.034100,0.688044,2025-01-04 18:16:58.921000+00:00
"The Field of AI Alignment: A Postmortem, and What To Do About It",0.035840,0.925483,2025-01-01 00:42:36.538000+00:00
Flower Crowns and Furry Mishaps by MyPalAI,0.037607,0.300000,2024-12-24 00:00:00+00:00
Preference Inversion,0.042690,0.633321,2025-01-03 23:49:06.168000+00:00
The Laws of Large Numbers,0.047607,0.540932,2025-01-04 18:06:02.387000+00:00
OpenAI Email Archives from Musk v. Altman,0.047660,0.700000,2024-11-01 00:00:00+00:00
debating buying NVDA in 2019,0.050691,0.526975,2025-01-04 18:07:35.832000+00:00


In [21]:
df2.select_dtypes(include=np.number).corr()['novelty'].sort_values()

improvement%_std    -0.482445
improvement_std     -0.457283
improvement%_mean   -0.442458
improvement%_max    -0.369768
improvement_max     -0.367002
before_max          -0.357355
improvement%_ppx    -0.264841
improvement_mean    -0.260877
after_max           -0.165648
improvement%_min    -0.080467
after_min           -0.000692
before_min           0.114827
improvement_min      0.179910
len                  0.222931
improvement_ppx      0.259635
before_std           0.367073
n_tokens             0.404075
before_mean          0.436573
before_ppx           0.454817
after_ppx            0.461407
after_std            0.464671
after_mean           0.470662
before_sum           0.574461
after_sum            0.580702
novelty              1.000000
Name: novelty, dtype: float64